In [1]:
from pprint import pprint as print

In [2]:
import pyarrow.dataset as ds

In [3]:
! ls ../data/chembl_36/exports/assay

assay-0.parquet


In [4]:
! ls ../data/chembl_36/exports/assay

assay-0.parquet


In [5]:
from pathlib import Path
path = Path("../data/chembl_36/exports/")

In [6]:
def ds_info(path:Path) -> None:
    dataset = ds.dataset(path, format="parquet")
    print(path)
    print(dataset.schema)
    print(dataset.head(10))

In [7]:
ds_info(path/"assay")

PosixPath('../data/chembl_36/exports/assay')
assay_id: int64
assay_type: string
assay_organism: string
assay_tax_id: int64
tid: int64
doc_id: int64
chembl_id: string
pyarrow.Table
assay_id: int64
assay_type: string
assay_organism: string
assay_tax_id: int64
tid: int64
doc_id: int64
chembl_id: string
----
assay_id: [[5,6,7,8,9,10,11,12,17,36]]
assay_type: [["F","F","F","F","F","F","F","F","A","A"]]
assay_organism: [["Homo sapiens","Homo sapiens","Mus musculus","Homo sapiens","Homo sapiens","Homo sapiens","Homo sapiens","Homo sapiens","Rattus norvegicus","Rattus norvegicus"]]
assay_tax_id: [[9606,9606,10090,9606,9606,9606,9606,9606,10116,10116]]
tid: [[80001,80001,80001,80001,80001,80001,80001,80001,100122,22224]]
doc_id: [[17430,17430,13799,17774,3801,17430,17430,17774,11347,11347]]
chembl_id: [["CHEMBL615121","CHEMBL615122","CHEMBL615123","CHEMBL615124","CHEMBL615125","CHEMBL615126","CHEMBL615127","CHEMBL615128","CHEMBL884521","CHEMBL615148"]]


In [8]:
ds_info(path/"activity")

PosixPath('../data/chembl_36/exports/activity')
activity_id: int64
assay_id: int64
molregno: int64
standard_type: string
standard_value: double
standard_units: string
standard_relation: string
standard_flag: int64
pchembl_value: double
type: string
value: double
units: string
potential_duplicate: int64
pyarrow.Table
activity_id: int64
assay_id: int64
molregno: int64
standard_type: string
standard_value: double
standard_units: string
standard_relation: string
standard_flag: int64
pchembl_value: double
type: string
value: double
units: string
potential_duplicate: int64
----
activity_id: [[31880,32640,32647,32706,32731,32992,33038,33404,33901,33908]]
assay_id: [[43319,103941,103941,147402,172926,79158,43319,77092,103941,103941]]
molregno: [[295513,1704117,152038,29061,172226,174476,107119,62917,151744,151328]]
standard_type: [["EC50","EC50","EC50","EC50","EC50","EC50","EC50","EC50","EC50","EC50"]]
standard_value: [[63.6,100000,4000,3000,280000,2511.89,90,33,4000,4000]]
standard_units: [["

# Spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


In [2]:
spark = SparkSession.builder \
    .appName("chembl-parquet") \
    .remote("sc://localhost:15002") \
    .getOrCreate()

In [3]:
df = spark.read.parquet("/data/chembl_36/exports/activity")


In [4]:
df.printSchema()

root
 |-- activity_id: long (nullable = true)
 |-- assay_id: long (nullable = true)
 |-- molregno: long (nullable = true)
 |-- standard_type: string (nullable = true)
 |-- standard_value: double (nullable = true)
 |-- standard_units: string (nullable = true)
 |-- standard_relation: string (nullable = true)
 |-- standard_flag: long (nullable = true)
 |-- pchembl_value: double (nullable = true)
 |-- type: string (nullable = true)
 |-- value: double (nullable = true)
 |-- units: string (nullable = true)
 |-- potential_duplicate: long (nullable = true)



In [5]:
df.show(10, truncate=False)
# Simple ops
print(f"rows = {df.count()}")

+-----------+--------+--------+-------------+--------------+--------------+-----------------+-------------+-------------+--------+------+-----+-------------------+
|activity_id|assay_id|molregno|standard_type|standard_value|standard_units|standard_relation|standard_flag|pchembl_value|type    |value |units|potential_duplicate|
+-----------+--------+--------+-------------+--------------+--------------+-----------------+-------------+-------------+--------+------+-----+-------------------+
|31880      |43319   |295513  |EC50         |63.6          |nM            |<                |1            |NULL         |EC50    |0.0636|uM   |0                  |
|32640      |103941  |1704117 |EC50         |100000.0      |nM            |>                |1            |NULL         |EC50    |100.0 |uM   |0                  |
|32647      |103941  |152038  |EC50         |4000.0        |nM            |>                |1            |NULL         |EC50    |4.0   |uM   |0                  |
|32706      |147

In [6]:
cols = ["value","units", "standard_value","standard_units"]

In [7]:
cols_df = df.select(*cols)
cols_df.show(10)
cols_df.count()

+------+-----+--------------+--------------+
| value|units|standard_value|standard_units|
+------+-----+--------------+--------------+
|0.0636|   uM|          63.6|            nM|
| 100.0|   uM|      100000.0|            nM|
|   4.0|   uM|        4000.0|            nM|
|3000.0|   nM|        3000.0|            nM|
| 280.0|   uM|      280000.0|            nM|
|  -5.6| NULL|       2511.89|            nM|
|  0.09|   uM|          90.0|            nM|
| 0.033|   uM|          33.0|            nM|
|   4.0|   uM|        4000.0|            nM|
|   4.0|   uM|        4000.0|            nM|
+------+-----+--------------+--------------+
only showing top 10 rows


3074840

In [10]:
filtered_df = cols_df.na.drop(subset=cols)
f_df= filtered_df
filtered_df.count()


2777652

In [36]:
def norm_vu(df) -> df:


    unit0 = F.lower(F.trim(F.col("units")))
    unit0 = F.regexp_replace(unit0, "[µμ]", "u")
    unit0 = F.regexp_replace(unit0, "micro", "u")
    unit0 = F.regexp_replace(unit0, "\\s+", "")  # remove spaces

    # exponent for formats like 10^-9M, 10'-9M, x10^-10M
    exp = F.regexp_extract(unit0, r"(?:x?10\^|10')([+-]?\d+)", 1)
    exp = F.when(exp != "", exp.cast("int")).otherwise(F.lit(0))
    
    base = F.regexp_replace(unit0, r"(?:x?10\^|10')([+-]?\d+)", "")
    
    base_mult = F.create_map(
        F.lit("m"), F.lit(1e9),
        F.lit("mol/l"), F.lit(1e9),
        F.lit("mm"), F.lit(1e6),
        F.lit("mmol/l"), F.lit(1e6),
        F.lit("um"), F.lit(1e3),
        F.lit("umol/l"), F.lit(1e3),
        F.lit("nm"), F.lit(1.0),
        F.lit("nmol/l"), F.lit(1.0),
        F.lit("pm"), F.lit(1e-3),
        F.lit("pmol/l"), F.lit(1e-3),
        F.lit("fm"), F.lit(1e-6),
        F.lit("nmol/ml"), F.lit(1e3),
        F.lit("pmol/ml"), F.lit(1.0),
        F.lit("umol/ml"), F.lit(1e6),
    )
    mult_to_nM = F.element_at(base_mult, base) * F.pow(F.lit(10.0), exp)

    return  df.withColumn("units_norm", unit0) \
            .withColumn("value_nM", (F.col("value") * mult_to_nM).cast("double"))
    
    return tmp_df


n_df = norm_vu(f_df)
n_df.distinct().show(10)

+-----------+-----+--------------+--------------+----------+------------------+
|      value|units|standard_value|standard_units|units_norm|          value_nM|
+-----------+-----+--------------+--------------+----------+------------------+
|     0.7459|   uM|         745.9|            nM|        um|             745.9|
|     0.0124|   uM|          12.4|            nM|        um|              12.4|
|      5.576|   uM|        5576.0|            nM|        um|            5576.0|
|      6.263|   um|        6263.0|            nM|        um|            6263.0|
|    0.03306|   um|         33.06|            nM|        um|             33.06|
|   20.77931|   um|      20779.31|            nM|        um|20779.309999999998|
|  1.5425817|   um|       1542.58|            nM|        um|         1542.5817|
|1.667177232|   um|       1667.18|            nM|        um|       1667.177232|
| 1.93650846|   um|       1936.51|            nM|        um|        1936.50846|
|3.422510947|   um|       3422.51|      

In [37]:
n_df.select("units_norm").distinct().show()

+----------+
|units_norm|
+----------+
|10^-7mol/l|
|10^-5mol/l|
|     ug/kg|
|   10^-5nm|
|   10^-10m|
|        fm|
|   nmol/ml|
|    nmol/l|
|    10'-6m|
|   10'-11m|
|    10'3nm|
|10^-4ug/ml|
|    ug/mol|
|         m|
|   10'-3um|
|    10^6nm|
|     mol/l|
|    10^-4m|
|        ul|
|10^-5ug/ml|
+----------+
only showing top 20 rows


In [ ]:
n

In [40]:
n_df.where("units_norm == 'nmol/l'").show()

+-------+------+--------------+--------------+----------+--------+
|  value| units|standard_value|standard_units|units_norm|value_nM|
+-------+------+--------------+--------------+----------+--------+
|   10.8|nmol/L|          10.8|            nM|    nmol/l|    10.8|
|   10.9|nmol/L|          10.9|            nM|    nmol/l|    10.9|
| 4600.0|nmol/L|        4600.0|            nM|    nmol/l|  4600.0|
|15000.0|nmol/L|       15000.0|            nM|    nmol/l| 15000.0|
|15000.0|nmol/L|       15000.0|            nM|    nmol/l| 15000.0|
|   77.2|nmol/L|          77.2|            nM|    nmol/l|    77.2|
|   56.5|nmol/L|          56.5|            nM|    nmol/l|    56.5|
|   22.3|nmol/L|          22.3|            nM|    nmol/l|    22.3|
|   50.4|nmol/L|          50.4|            nM|    nmol/l|    50.4|
| 4500.0|nmol/L|        4500.0|            nM|    nmol/l|  4500.0|
|15000.0|nmol/L|       15000.0|            nM|    nmol/l| 15000.0|
|15000.0|nmol/L|       15000.0|            nM|    nmol/l| 1500

In [43]:
n_df.count()

2777652

In [44]:
n_df.na.drop().count()

2724064

In [86]:
n_df.groupBy("standard_units").count().show(1000)

+--------------+-------+
|standard_units|  count|
+--------------+-------+
|      10'20 uM|      1|
|         ug/kg|      2|
|        10'6pM|      3|
|        10'3pM|      1|
|            uL|     27|
|   10^-6 mol/L|     18|
|        ug/mol|     11|
|        10^3nM|     54|
|   10^-3umol/L|      1|
|      10^-6 uM|      3|
|        pmol/L|     24|
|       nmol/Kg|      6|
|      10^-5 uM|     16|
|         uL/ml|      1|
|        10'5pM|      5|
|        10'6uM|      2|
|   10^-6 ug/ml|      1|
|   10^-10mol/L|      9|
|        microA|      1|
|   10^-5 ug/ml|      2|
|    10^-8mol/L|     60|
|   10^-4microM|     21|
|       ug.mL-1|  49001|
|    10^-9mol/L|     74|
|    10^-7mol/L|     81|
|   10^-5 mol/L|     47|
|       10^-4nM|     10|
|          ug/g|      5|
|     10^-8nmol|      2|
|           ppm|      7|
|    10^-4mg/ml|      1|
|       10^3 uM|      9|
|            ug|    190|
|             %|    179|
|       10^2 nM|      6|
|        10'5uM|      1|
|     10^-6nmol|      3|


In [75]:
n_df.filter(F.abs(F.col("standard_value") - F.col("value_nM")) >= 1e-2).show()

+-----+-------+--------------+--------------+----------+------------------+
|value|  units|standard_value|standard_units|units_norm|          value_nM|
+-----+-------+--------------+--------------+----------+------------------+
|  1.0| 10'8pM|           1.0|        10'8pM|    10'8pm|          100000.0|
|  5.8| 10'7pM|           5.8|        10'7pM|    10'7pm|           58000.0|
|  5.7| 10'6pM|           5.7|        10'6pM|    10'6pm|            5700.0|
|  1.0| 10'7pM|           1.0|        10'7pM|    10'7pm|           10000.0|
|  4.2| 10'7pM|           4.2|        10'7pM|    10'7pm|           42000.0|
|  1.0| 10'8pM|           1.0|        10'8pM|    10'8pm|          100000.0|
|  1.7| 10'5pM|           1.7|        10'5pM|    10'5pm|             170.0|
|  1.0| 10'8pM|           1.0|        10'8pM|    10'8pm|          100000.0|
|  1.0| 10'8pM|           1.0|        10'8pM|    10'8pm|          100000.0|
|  1.0| 10'8pM|           1.0|        10'8pM|    10'8pm|          100000.0|
|  2.0| 10'5

In [81]:
n_df.filter(F.abs(F.col("standard_value") - F.col("value_nM")) >= 1e-3).filter("standard_units == 'nM'").show()

+-----------+-----+--------------+--------------+----------+------------------+
|      value|units|standard_value|standard_units|units_norm|          value_nM|
+-----------+-----+--------------+--------------+----------+------------------+
|0.288428703|   uM|        288.43|            nM|        um|        288.428703|
|2.505985648|   uM|       2505.99|            nM|        um|       2505.985648|
|0.513984494|   uM|        513.98|            nM|        um|        513.984494|
|0.823374116|   uM|        823.37|            nM|        um|        823.374116|
|0.060321109|   uM|         60.32|            nM|        um|         60.321109|
|0.836665417|   uM|        836.67|            nM|        um| 836.6654169999999|
| 0.31774596|   uM|        317.75|            nM|        um|317.74595999999997|
|0.472501052|   uM|         472.5|            nM|        um|472.50105199999996|
|1.482348544|   uM|       1482.35|            nM|        um|       1482.348544|
| 0.09528385|   uM|         95.28|      

Almost all differences are just floating point numbers
only for 1e-2 we have difference results but those are for wierd standard_units such us 10'8pm.

We should be safe with just taking nM

In [66]:
def pIC(df):
        -F.log10(F.col("standard_value") * 1e9)
    return -math.log10(molar)

In [85]:
def add_pIC(df):
    # keep only nM rows, ignore non-positive values to avoid log issues
    return (
        df.filter((F.col("standard_units") == "nM") & (F.col("standard_value") > 0))
          .withColumn("pIC", -F.log10(F.col("standard_value") * F.lit(1e-9)))
    )

# example
# df_with_pic = add_pIC(df)
# df_with_pic.select("standard_value", "standard_units", "pIC").show(5)
add_pIC(n_df).show()

+------+-----+--------------+--------------+------------------+
| value|units|standard_value|standard_units|               pIC|
+------+-----+--------------+--------------+------------------+
|0.0636|   uM|          63.6|            nM| 7.196542884351586|
| 100.0|   uM|      100000.0|            nM|               4.0|
|   4.0|   uM|        4000.0|            nM|5.3979400086720375|
|3000.0|   nM|        3000.0|            nM| 5.522878745280337|
| 280.0|   uM|      280000.0|            nM|3.5528419686577806|
|  0.09|   uM|          90.0|            nM| 7.045757490560675|
| 0.033|   uM|          33.0|            nM| 7.481486060122112|
|   4.0|   uM|        4000.0|            nM|5.3979400086720375|
|   4.0|   uM|        4000.0|            nM|5.3979400086720375|
|   9.7|   uM|        9700.0|            nM| 5.013228265733755|
|1000.0|   uM|     1000000.0|            nM|               3.0|
|   4.8|   uM|        4800.0|            nM| 5.318758762624412|
|  23.0|   uM|       23000.0|           